## Initialization

In [1]:
import pandas as pd
import numpy as np
import keras

Using Theano backend.


## Importing the Data

In [2]:
dataset = pd.read_csv('Seismic Bumps.csv', header = None)
dataset.shape

(2584, 19)

In [3]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,a,a,N,15180,48,-72,-72,a,0,0,0,0,0,0,0,0,0,0,0
1,a,a,N,14720,33,-70,-79,a,1,0,1,0,0,0,0,0,2000,2000,0
2,a,a,N,8050,30,-81,-78,a,0,0,0,0,0,0,0,0,0,0,0
3,a,a,N,28820,171,-23,40,a,1,0,1,0,0,0,0,0,3000,3000,0
4,a,a,N,12640,57,-63,-52,a,0,0,0,0,0,0,0,0,0,0,0


## Create X and Y

In [4]:
X = dataset.iloc[:, 0:18].values
Y = dataset.iloc[:, 18].values

In [5]:
X.shape

(2584, 18)

In [6]:
Y.shape

(2584,)

In [7]:
X

array([['a', 'a', 'N', ..., 0, 0, 0],
       ['a', 'a', 'N', ..., 0, 2000, 2000],
       ['a', 'a', 'N', ..., 0, 0, 0],
       ..., 
       ['b', 'a', 'W', ..., 0, 0, 0],
       ['a', 'a', 'W', ..., 0, 0, 0],
       ['a', 'a', 'W', ..., 0, 0, 0]], dtype=object)

In [8]:
Y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Preprocess the Data

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [10]:
le_Y = LabelEncoder()

In [11]:
Y = le_Y.fit_transform(Y)
Y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [12]:
def encoder_X(index):
    le_X = LabelEncoder()
    X[:, index] = le_X.fit_transform(X[:, index])

In [13]:
to_be_encoded_indices = [0, 1, 2, 7]

In [14]:
for x in to_be_encoded_indices:
    encoder_X(x)

In [15]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 2000, 2000],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=object)

In [16]:
ohe_X = OneHotEncoder(categorical_features = [7])

In [17]:
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]
X.shape

(2584, 19)

In [18]:
ohe_X = OneHotEncoder(categorical_features = [3])

In [19]:
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]
X.shape

(2584, 20)

In [20]:
sc_X = StandardScaler()

In [21]:
X = sc_X.fit_transform(X)
X

array([[-0.76630515, -0.13757705, -0.29895828, ...,  0.        ,
        -0.24332671, -0.22108685],
       [-0.76630515, -0.13757705, -0.29895828, ...,  0.        ,
        -0.14551225, -0.11774749],
       [-0.76630515, -0.13757705, -0.29895828, ...,  0.        ,
        -0.24332671, -0.22108685],
       ..., 
       [-0.76630515, -0.13757705, -0.29895828, ...,  0.        ,
        -0.24332671, -0.22108685],
       [-0.76630515, -0.13757705, -0.29895828, ...,  0.        ,
        -0.24332671, -0.22108685],
       [-0.76630515, -0.13757705, -0.29895828, ...,  0.        ,
        -0.24332671, -0.22108685]])

## Create Train and Test Data

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [24]:
X_train.shape

(2067, 20)

In [25]:
X_test.shape

(517, 20)

In [26]:
Y_train.shape

(2067,)

In [27]:
Y_test.shape

(517,)

In [28]:
pd.DataFrame(pd.DataFrame(Y_train)[0].value_counts())

,0
0,1935
1,132


In [29]:
pd.DataFrame(pd.DataFrame(Y_test)[0].value_counts())

,0
0,479
1,38


## Create and train the ANN Classifier

In [30]:
from keras.models import Sequential
from keras.layers import Dense

In [46]:
clf_ann = Sequential()

# 10 20 40 80 160
# 40 40 160

# First Hidden Layer
clf_ann.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu', input_dim = 20))

# Second Hidden Layer
#clf_ann.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu'))

clf_ann.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu'))

#clf_ann.add(Dense(output_dim = 80, init = 'uniform', activation = 'relu'))

clf_ann.add(Dense(output_dim = 160, init = 'uniform', activation = 'relu'))

# Output Layer
clf_ann.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compile the ANN
clf_ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN on the Train Data
clf_ann.fit(X_train, Y_train, batch_size = 10, nb_epoch = 100)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=40, input_dim=20, activation="relu")`
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=40, activation="relu")`
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=160, activation="relu")`
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=1, activation="sigmoid")`
C:\Users\user\Anaconda3\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
2067/2067 [==============================] - 0s - loss: 0.2870 - acc: 0.9361     
Epoch 2/100
2067/2067 [==============================] - 0s - loss: 0.2094 - acc: 0.9361     
Epoch 3/100
2067/2067 [==============================] - 0s - loss: 0.2083 - acc: 0.9361     
Epoch 4/100
2067/2067 [==============================] - 0s - loss: 0.2040 - acc: 0.9361     
Epoch 5/100
2067/2067 [==============================] - 0s - loss: 0.2041 - acc: 0.9361     
Epoch 6/100
2067/2067 [==============================] - 0s - loss: 0.2034 - acc: 0.9361     
Epoch 7/100
2067/2067 [==============================] - 0s - loss: 0.2020 - acc: 0.9361     
Epoch 8/100
2067/2067 [==============================] - 0s - loss: 0.1993 - acc: 0.9361     
Epoch 9/100
2067/2067 [==============================] - 0s - loss: 0.1970 - acc: 0.9371     
Epoch 10/100
2067/2067 [==============================] - 0s - loss: 0.1993 - acc: 0.9366     
Epoch 11/100
2067/2067 [==============================] - 0

2067/2067 [==============================] - 0s - loss: 0.0875 - acc: 0.9719     
Epoch 87/100
2067/2067 [==============================] - 0s - loss: 0.0926 - acc: 0.9700     
Epoch 88/100
2067/2067 [==============================] - 0s - loss: 0.0844 - acc: 0.9729     
Epoch 89/100
2067/2067 [==============================] - 0s - loss: 0.0834 - acc: 0.9739     
Epoch 90/100
2067/2067 [==============================] - 0s - loss: 0.0854 - acc: 0.9724     - ETA: 0s - loss: 0.0855 - acc: 0.97
Epoch 91/100
2067/2067 [==============================] - 0s - loss: 0.0837 - acc: 0.9710     
Epoch 92/100
2067/2067 [==============================] - 0s - loss: 0.0904 - acc: 0.9700     
Epoch 93/100
2067/2067 [==============================] - 0s - loss: 0.1034 - acc: 0.9642     
Epoch 94/100
2067/2067 [==============================] - 0s - loss: 0.0903 - acc: 0.9719     
Epoch 95/100
2067/2067 [==============================] - 0s - loss: 0.0907 - acc: 0.9681     
Epoch 96/100
2067/2067 [===

In [47]:
# Test the ANN on the Test Data
Y_pred = clf_ann.predict(X_test)
Y_pred = (Y_pred > 0.5)

## Check the Accuracy

In [48]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [49]:
accuracy_score(Y_pred, Y_test)

0.90715667311411996

In [50]:
confusion_matrix(Y_pred, Y_test)

array([[461,  30],
       [ 18,   8]], dtype=int64)